<a href="https://colab.research.google.com/github/l43lu/NLP/blob/master/German_News_Articles_Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install stopwordsiso

In [2]:
!pip install nlpaug

In [3]:
!pip install transformers

In [4]:
import stopwordsiso as stopwords
import unicodedata as unicode

In [5]:
#%tensorflow_version 2.x
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

import os
import errno
import io
tf.__version__

from os import listdir
from os.path import isfile, join, isdir

In [6]:
import nlpaug.augmenter.word as naw

In [7]:
import pandas as pd

In [8]:
test  = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/test.csv', sep=';', names=['sub','text_1','text_2'])
train = pd.read_csv('https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv', sep=';', names=['sub','text'])

In [9]:
test.fillna('', inplace=True)
train.fillna('', inplace=True)

In [10]:
test['text'] = test['text_1'] + test['text_2']

In [11]:
test.drop(['text_1','text_2'],inplace=True, axis=1)

In [12]:
test.head()

,sub,text
0,Wirtschaft,"'Die Gewerkschaft GPA-djp lanciert den ""All-in..."
1,Sport,Franzosen verteidigen 2:1-Führung – Kritische ...
2,Web,'Neues Video von Designern macht im Netz die R...
3,Sport,23-jähriger Brasilianer muss vier Spiele pausi...
4,International,Aufständische verwendeten Chemikalie bei Gefec...


In [13]:
text_type_train_list = train['sub'].unique()
text_type_test_list = test['sub'].unique()

In [14]:
count_files_in_folder = {}
inverse_count_files_in_folder = {}
for text_type in text_type_train_list:
    count_text_type = len(train.loc[train['sub'] == text_type])
    
    count_files_in_folder[text_type] = count_text_type
    inverse_count_files_in_folder[count_text_type] = text_type

In [15]:
all_values = count_files_in_folder.values()
max_value = max(all_values)
max_value

1510

In [16]:
del count_files_in_folder[inverse_count_files_in_folder[max_value]]

In [17]:
### get diff to maxvalue for subjects
for key in count_files_in_folder:
    count_files_in_folder[key] = max_value - count_files_in_folder[key]

In [18]:
count_files_in_folder

{'Etat': 909,
 'Inland': 597,
 'International': 150,
 'Kultur': 1025,
 'Sport': 429,
 'Web': 1,
 'Wirtschaft': 240,
 'Wissenschaft': 994}

## **Backtranslation**

In [21]:
def generate_back_translation(df,subject_list=[],subject_to_generate_list=[], sub_column_name='sub', text_column_name='text'):
    for i, subject in enumerate(subject_list):
        amount_to_generate = subject_to_generate_list[i]
        sub_df             = df.loc[df[sub_column_name] == subject]
        
        for index, row in sub_df.iloc[0:amount_to_generate].iterrows():
            current_text = row['text']
            text_aug     = back_translation_aug.augment(current_text)
           # print("=============")
           # print("Current Text::::", current_text)
           # print("*******************************")
           # print("Backtranslated Text:", text_aug)
            new_row = {sub_column_name:subject, text_column_name:text_aug}
            df = df.append(new_row, ignore_index=True)
            

    
    return df

In [23]:
counts = list(count_files_in_folder.values())
subs   = list(count_files_in_folder.keys())

In [19]:
back_translation_aug = naw.BackTranslationAug(
    from_model_name='facebook/wmt19-de-en',
    to_model_name='facebook/wmt19-en-de',
    #device='cuda'
)

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/825 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/308k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/829k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/308k [00:00<?, ?B/s]

In [ ]:
return_df = generate_back_translation(train,subs,counts)

Token indices sequence length is longer than the specified maximum sequence length for this model (1163 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
return_df.to_csv('backtranslated_train.csv', sep=';')